In [1]:
import pandas as pd
import numpy as np
from scipy import signal
from tqdm import tqdm
from numpy.fft import fft, fftshift
import random
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
import os
from sklearn.preprocessing import StandardScaler
from numpy.random import seed

In [3]:
# data load
# path 변수를 적절히 변경
x_train_path = os.path.join('data/train_features.csv')
y_train_path = os.path.join('data/train_labels.csv')
x_test_path = os.path.join('data/test_features.csv')
sub_path = os.path.join('data/sample_submission.csv')

train = pd.read_csv(x_train_path)
train_label = pd.read_csv(y_train_path)
test = pd.read_csv(x_test_path)
sub = pd.read_csv(sub_path)

In [4]:
train['acc_Energy']=(train['acc_x']**2+train['acc_y']**2+train['acc_z']**2)**(1/3)
test['acc_Energy']=(test['acc_x']**2+test['acc_y']**2+test['acc_z']**2)**(1/3)

train['gy_Energy']=(train['gy_x']**2+train['gy_y']**2+train['gy_z']**2)**(1/3)
test['gy_Energy']=(test['gy_x']**2+test['gy_y']**2+test['gy_z']**2)**(1/3)

train['gy_acc_Energy']=((train['gy_x']-train['acc_x'])**2+(train['gy_y']-train['acc_y'])**2+(train['gy_z']-train['acc_z'])**2)**(1/3)
test['gy_acc_Energy']=((test['gy_x']-test['acc_x'])**2+(test['gy_y']-test['acc_y'])**2+(test['gy_z']-test['acc_z'])**2)**(1/3)

In [5]:
dt=0.02 
def jerk_signal(signal): 
        return np.array([(signal[i+1]-signal[i])/dt for i in range(len(signal)-1)])

In [6]:
train_dt=[]
for i in tqdm(train['id'].unique()):
    temp=train.loc[train['id']==i]
    for v in train.columns[2:]:
        values=jerk_signal(temp[v].values)
        values=np.insert(values,0,0)
        temp.loc[:,v+'_dt']=values
    train_dt.append(temp)

100%|██████████████████████████████████████| 3125/3125 [00:27<00:00, 114.54it/s]


In [7]:
test_dt=[]
for i in tqdm(test['id'].unique()):
    temp=test.loc[test['id']==i]
    for v in train.columns[2:]:
        values=jerk_signal(temp[v].values)
        values=np.insert(values,0,0)
        temp.loc[:,v+'_dt']=values
    test_dt.append(temp)

100%|████████████████████████████████████████| 782/782 [00:06<00:00, 127.91it/s]


In [8]:
from scipy import fftpack
from numpy.fft import *

def fourier_transform_one_signal(t_signal):
    complex_f_signal= fftpack.fft(t_signal)
    amplitude_f_signal=np.abs(complex_f_signal)
    return amplitude_f_signal

In [9]:
train=pd.concat(train_dt)

In [10]:
fft=[]
for i in tqdm(train['id'].unique()):
    temp=train.loc[train['id']==i]
    for i in train.columns[2:8]:
        temp[i]=fourier_transform_one_signal(temp[i].values)
    fft.append(temp)
train=pd.concat(fft)

100%|██████████████████████████████████████| 3125/3125 [00:05<00:00, 533.19it/s]


In [11]:
test=pd.concat(test_dt)

In [12]:
fft_t=[]
for i in tqdm(test['id'].unique()):
    temp=test.loc[test['id']==i]
    for i in test.columns[2:8]:
        temp[i]=fourier_transform_one_signal(temp[i].values)
    fft_t.append(temp)
test=pd.concat(fft_t)

100%|████████████████████████████████████████| 782/782 [00:00<00:00, 893.59it/s]


In [13]:
col=train.columns
train_s=train.copy()
test_s=test.copy()

In [14]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler

scaler = StandardScaler()

train_s.iloc[:,2:]= scaler.fit_transform(train_s.iloc[:,2:])
train_sc = pd.DataFrame(data = train_s,columns =col)

test_s.iloc[:,2:]= scaler.transform(test_s.iloc[:,2:])
test_sc = pd.DataFrame(data = test_s,columns =col)

In [15]:
# make dataset
# x를 시계열 데이터로 변경 시켜주는 함수
def make_dataset(data):
  ids = data['id'].unique()
  id_data = data.groupby('id')
  series_data = []

  for i in ids:
    df = id_data.get_group(i)
    df = df.drop(['id', 'time'], axis=1)
    series_data.append(df.to_numpy())

  series_data = np.array(series_data)
  return series_data

In [16]:
series_train = make_dataset(train_sc)
series_test = make_dataset(test_sc)

In [17]:
series_train.shape, series_test.shape

((3125, 600, 18), (782, 600, 18))

In [18]:
# label one-hot 인코딩
labels = train_label['label'].to_numpy()

In [19]:
import tensorflow as tf

2021-09-30 14:08:01.272100: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [20]:
# dataset과 validation set을 만들어 주는 함수
# validation set은 shuffle 적용 x
def make_train(series_data, labels):
  cat_y = tf.keras.utils.to_categorical(labels)

  BATCH_SIZE = 64
  train_dataset = tf.data.Dataset.from_tensor_slices((series_data, cat_y))
  train_dataset = train_dataset.batch(BATCH_SIZE)
  train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

  return train_dataset

def make_val(series_data, labels):
  cat_y = tf.keras.utils.to_categorical(labels)

  BATCH_SIZE = 64
  val_dataset = tf.data.Dataset.from_tensor_slices((series_data, cat_y))
  val_dataset = val_dataset.batch(BATCH_SIZE)
  val_dataset = val_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

  return train_dataset

In [21]:
from tensorflow import keras
from tensorflow.keras import backend as K

In [36]:
# 가중치 적용 loss
# https://openaccess.thecvf.com/content_CVPR_2019/papers/Cui_Class-Balanced_Loss_Based_on_Effective_Number_of_Samples_CVPR_2019_paper.pdf
# 위 논문 참고
def weighted_loss(labels, y_pred, beta=0.9):
        no_samples = [12, 21, 20, 23, 35, 25, 24, 26, 97, 37, 20, 23, 12,
                     12, 25, 25, 22, 27, 47, 20, 26, 27, 19, 20, 35, 24,
                     1518, 34, 55, 20, 35, 20, 18, 20, 22, 30, 28, 35, 20,
                     20, 34, 20, 20, 35, 21, 22, 20, 26, 25, 30, 37, 24,
                     12, 13, 23, 37, 36, 20, 20, 23, 48]

        weight = tf.divide(1-beta, 1-tf.pow(beta, no_samples))
        #y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        loss = labels * K.log(y_pred) * weight
        loss = -K.sum(loss, -1)
        return loss

def focal(labels, y_pred, gamma=2):
    #y_pred = tf.nn.softmax(logits, dim=-1)  # [batch_size, num_classes]
    #labels = tf.squeeze(labels)  # label example: [0,1,2,3]
    #labels = tf.to_float(tf.one_hot(labels, depth=y_pred.shape[1]))

    loss = -labels * ((1 - y_pred) ** gamma) * K.log(y_pred)
    loss = tf.reduce_mean(tf.reduce_sum(loss, axis=1))
    return loss


# 모델을 만들어 주는 함수
def base():
    seed(2021)
    tf.random.set_seed(2021)
    model = keras.models.Sequential([
            keras.layers.Conv1D(filters=128, kernel_size=9, padding='same', input_shape=[600, 18]),
            keras.layers.BatchNormalization(),
            keras.layers.Activation('relu'),
            keras.layers.Dropout(0.3),
            keras.layers.Conv1D(filters=256, kernel_size=6, padding='same'),
            keras.layers.BatchNormalization(),
            keras.layers.Activation('relu'),
            keras.layers.Dropout(0.4),
            keras.layers.Conv1D(filters=128, kernel_size=3,padding='same'),
            keras.layers.BatchNormalization(),
            keras.layers.Activation('relu'),
            keras.layers.Dropout(0.5),
            keras.layers.GlobalAveragePooling1D(),
            keras.layers.Dense(61, activation='softmax')
    ])
    model.compile(optimizer='adam',
                loss=focal, 
                metrics=['accuracy'])
    return model

In [37]:
# checkpoint path
# 중간중간 모델의 weight를 저장할 경로 설정
ckpt_name = 'fold_cnn_weighted_ckpt.hdf5'
checkpoint_dir_path = os.path.join('checkpoint')
checkpoint_path = os.path.join(checkpoint_dir_path, ckpt_name)

# callback 함수 목록
callbacks_list = [
    # 매 epoch 마다 val_loss를 체크하여 가장 낮은 상태의 weight를 저장
    tf.keras.callbacks.ModelCheckpoint(
        filepath = checkpoint_path,
        monitor='val_loss',
        mode='min',
        save_weights_only=True,
        save_best_only=True
    ),
    # 8번 동안 val_loss의 향상이 없으면 훈련 종료
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        mode='min',
        verbose=1, 
        patience=8
    ),
    tf.keras.callbacks.ReduceLROnPlateau(patience = 4,verbose = 1,factor = 0.5) 
]

In [38]:
from sklearn.model_selection import StratifiedKFold

In [39]:
# k-fold 적용
# 데이터 수가 적어 10개의 fold 사용
k = 10 
split = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
results = []    # 매 fold 훈련 후 loss 저장
models = []     # 매 fold 훈련 후 해당 모델 저장
accs = []
for i, (train, val) in enumerate(split.split(series_train, labels)):
    print('Fold', i)
    print('#'*20)

    train_dataset = make_train(series_train[train], labels[train])
    val_dataset = make_val(series_train[val], labels[val])

    model = base()

    model.fit(train_dataset, validation_data=val_dataset, callbacks=callbacks_list, epochs=100)

    # 모델 복원
    model.load_weights(checkpoint_path)
    result = model.evaluate(val_dataset)[0]
    acc = model.evaluate(val_dataset)[1]
    
    results.append(result)
    accs.append(acc)
    models.append(model)

    del model, train_dataset, val_dataset

Fold 0
####################
Epoch 1/100
44/44 [==============================] - 1s 32ms/step - loss: 2.5263 - accuracy: 0.4587 - val_loss: 3.1265 - val_accuracy: 0.2742
Epoch 2/100
44/44 [==============================] - 1s 28ms/step - loss: 1.6740 - accuracy: 0.5512 - val_loss: 1.6070 - val_accuracy: 0.5523
Epoch 3/100
44/44 [==============================] - 1s 28ms/step - loss: 1.4278 - accuracy: 0.5853 - val_loss: 1.5644 - val_accuracy: 0.5605
Epoch 4/100
44/44 [==============================] - 1s 28ms/step - loss: 1.2239 - accuracy: 0.6266 - val_loss: 1.4011 - val_accuracy: 0.5693
Epoch 5/100
44/44 [==============================] - 1s 29ms/step - loss: 1.0585 - accuracy: 0.6554 - val_loss: 1.2118 - val_accuracy: 0.6070
Epoch 6/100
44/44 [==============================] - 1s 29ms/step - loss: 0.9112 - accuracy: 0.6863 - val_loss: 1.0517 - val_accuracy: 0.6362
Epoch 7/100
44/44 [==============================] - 1s 27ms/step - loss: 0.7806 - accuracy: 0.7194 - val_loss: 0.9278 -

44/44 [==============================] - 1s 25ms/step - loss: 0.0547 - accuracy: 0.9687 - val_loss: 0.1353 - val_accuracy: 0.9243
Epoch 58/100
43/44 [============================>.] - ETA: 0s - loss: 0.0520 - accuracy: 0.9720
Epoch 00058: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
44/44 [==============================] - 1s 27ms/step - loss: 0.0518 - accuracy: 0.9723 - val_loss: 0.1273 - val_accuracy: 0.9310
Epoch 59/100
44/44 [==============================] - 1s 27ms/step - loss: 0.0460 - accuracy: 0.9751 - val_loss: 0.0906 - val_accuracy: 0.9445
Epoch 60/100
44/44 [==============================] - 1s 29ms/step - loss: 0.0433 - accuracy: 0.9787 - val_loss: 0.0827 - val_accuracy: 0.9467
Epoch 61/100
44/44 [==============================] - 1s 29ms/step - loss: 0.0370 - accuracy: 0.9833 - val_loss: 0.0782 - val_accuracy: 0.9431
Epoch 62/100
44/44 [==============================] - 1s 29ms/step - loss: 0.0375 - accuracy: 0.9815 - val_loss: 0.0753 - val_accuracy:

44/44 [==============================] - 1s 27ms/step - loss: 0.1757 - accuracy: 0.9072 - val_loss: 0.2480 - val_accuracy: 0.8741
Epoch 28/100
44/44 [==============================] - 1s 26ms/step - loss: 0.1664 - accuracy: 0.9147 - val_loss: 0.2193 - val_accuracy: 0.8894
Epoch 29/100
44/44 [==============================] - 1s 26ms/step - loss: 0.1564 - accuracy: 0.9154 - val_loss: 0.2198 - val_accuracy: 0.8805
Epoch 30/100
44/44 [==============================] - 1s 27ms/step - loss: 0.1528 - accuracy: 0.9171 - val_loss: 0.2199 - val_accuracy: 0.8805
Epoch 31/100
44/44 [==============================] - 1s 27ms/step - loss: 0.1501 - accuracy: 0.9147 - val_loss: 0.2533 - val_accuracy: 0.8741
Epoch 32/100
44/44 [==============================] - 1s 27ms/step - loss: 0.1487 - accuracy: 0.9211 - val_loss: 0.1982 - val_accuracy: 0.8905
Epoch 33/100
44/44 [==============================] - 1s 27ms/step - loss: 0.1408 - accuracy: 0.9203 - val_loss: 0.2199 - val_accuracy: 0.8791
Epoch 34/100

Epoch 81/100
44/44 [==============================] - 1s 27ms/step - loss: 0.0208 - accuracy: 0.9918 - val_loss: 0.0691 - val_accuracy: 0.9555
Epoch 82/100
43/44 [============================>.] - ETA: 0s - loss: 0.0199 - accuracy: 0.9942
Epoch 00082: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.
44/44 [==============================] - 1s 26ms/step - loss: 0.0198 - accuracy: 0.9943 - val_loss: 0.0626 - val_accuracy: 0.9627
Epoch 83/100
44/44 [==============================] - 1s 29ms/step - loss: 0.0182 - accuracy: 0.9961 - val_loss: 0.0568 - val_accuracy: 0.9673
Epoch 84/100
44/44 [==============================] - 1s 27ms/step - loss: 0.0185 - accuracy: 0.9957 - val_loss: 0.0583 - val_accuracy: 0.9659
Epoch 85/100
44/44 [==============================] - 1s 26ms/step - loss: 0.0186 - accuracy: 0.9947 - val_loss: 0.0581 - val_accuracy: 0.9662
Epoch 86/100
44/44 [==============================] - 1s 28ms/step - loss: 0.0190 - accuracy: 0.9932 - val_loss: 0.0573 - v

44/44 [==============================] - 1s 27ms/step - loss: 0.1336 - accuracy: 0.9307 - val_loss: 0.1727 - val_accuracy: 0.8969
Epoch 34/100
44/44 [==============================] - 1s 27ms/step - loss: 0.1275 - accuracy: 0.9321 - val_loss: 0.1591 - val_accuracy: 0.9015
Epoch 35/100
44/44 [==============================] - 1s 26ms/step - loss: 0.1245 - accuracy: 0.9310 - val_loss: 0.1524 - val_accuracy: 0.9029
Epoch 36/100
44/44 [==============================] - 1s 26ms/step - loss: 0.1140 - accuracy: 0.9331 - val_loss: 0.1679 - val_accuracy: 0.8951
Epoch 37/100
44/44 [==============================] - 1s 27ms/step - loss: 0.1100 - accuracy: 0.9381 - val_loss: 0.1490 - val_accuracy: 0.9061
Epoch 38/100
44/44 [==============================] - 1s 26ms/step - loss: 0.1055 - accuracy: 0.9403 - val_loss: 0.1440 - val_accuracy: 0.9221
Epoch 39/100
44/44 [==============================] - 1s 26ms/step - loss: 0.1018 - accuracy: 0.9399 - val_loss: 0.1317 - val_accuracy: 0.9257
Epoch 40/100

...

Epoch 99/100
44/44 [==============================] - 1s 26ms/step - loss: 0.0156 - accuracy: 0.9982 - val_loss: 0.0548 - val_accuracy: 0.9684
Epoch 100/100
44/44 [==============================] - 0s 5ms/step - loss: 0.0676 - accuracy: 0.9662
Fold 7
####################
Epoch 1/100
44/44 [==============================] - 1s 28ms/step - loss: 2.5151 - accuracy: 0.4621 - val_loss: 2.8855 - val_accuracy: 0.3466
Epoch 2/100
44/44 [==============================] - 1s 26ms/step - loss: 1.6621 - accuracy: 0.5560 - val_loss: 1.5931 - val_accuracy: 0.5603
Epoch 3/100
44/44 [==============================] - 1s 27ms/step - loss: 1.4203 - accuracy: 0.5930 - val_loss: 1.5416 - val_accuracy: 0.5617
Epoch 4/100
44/44 [==============================] - 1s 25ms/step - loss: 1.2164 - accuracy: 0.6306 - val_loss: 1.3984 - val_accuracy: 0.5681
Epoch 5/100
44/44 [==============================] - 1s 25ms/step - loss: 1.0543 - accuracy: 0.6623 - val_loss: 1.2195 - val_accuracy: 0.6065
Epoch 6/100
44/44 

Epoch 54/100
44/44 [==============================] - 1s 26ms/step - loss: 0.0462 - accuracy: 0.9744 - val_loss: 0.0922 - val_accuracy: 0.9467
Epoch 55/100
44/44 [==============================] - 1s 26ms/step - loss: 0.0434 - accuracy: 0.9787 - val_loss: 0.0848 - val_accuracy: 0.9534
Epoch 56/100
44/44 [==============================] - 1s 26ms/step - loss: 0.0422 - accuracy: 0.9808 - val_loss: 0.0923 - val_accuracy: 0.9545
Epoch 57/100
44/44 [==============================] - 1s 27ms/step - loss: 0.0512 - accuracy: 0.9698 - val_loss: 0.1087 - val_accuracy: 0.9403
Epoch 58/100
44/44 [==============================] - 1s 26ms/step - loss: 0.0475 - accuracy: 0.9737 - val_loss: 0.0887 - val_accuracy: 0.9549
Epoch 59/100
44/44 [==============================] - 1s 26ms/step - loss: 0.0425 - accuracy: 0.9762 - val_loss: 0.0842 - val_accuracy: 0.9499
Epoch 60/100
44/44 [==============================] - 1s 26ms/step - loss: 0.0388 - accuracy: 0.9794 - val_loss: 0.0745 - val_accuracy: 0.9573

44/44 [==============================] - 1s 25ms/step - loss: 0.2298 - accuracy: 0.8855 - val_loss: 0.2484 - val_accuracy: 0.8752
Epoch 22/100
44/44 [==============================] - 1s 27ms/step - loss: 0.2268 - accuracy: 0.8798 - val_loss: 0.2389 - val_accuracy: 0.8759
Epoch 23/100
44/44 [==============================] - 1s 26ms/step - loss: 0.2118 - accuracy: 0.8923 - val_loss: 0.2380 - val_accuracy: 0.8809
Epoch 24/100
44/44 [==============================] - 1s 27ms/step - loss: 0.1982 - accuracy: 0.8955 - val_loss: 0.2362 - val_accuracy: 0.8806
Epoch 25/100
44/44 [==============================] - 1s 27ms/step - loss: 0.1898 - accuracy: 0.8962 - val_loss: 0.2315 - val_accuracy: 0.8759
Epoch 26/100
44/44 [==============================] - 1s 26ms/step - loss: 0.1793 - accuracy: 0.8994 - val_loss: 0.2219 - val_accuracy: 0.8763
Epoch 27/100
44/44 [==============================] - 1s 26ms/step - loss: 0.1729 - accuracy: 0.9058 - val_loss: 0.2441 - val_accuracy: 0.8731
Epoch 28/100

44/44 [==============================] - 1s 25ms/step - loss: 0.6032 - accuracy: 0.7629 - val_loss: 0.6783 - val_accuracy: 0.7323
Epoch 10/100
44/44 [==============================] - 1s 26ms/step - loss: 0.5388 - accuracy: 0.7757 - val_loss: 0.6069 - val_accuracy: 0.7576
Epoch 11/100
44/44 [==============================] - 1s 26ms/step - loss: 0.4852 - accuracy: 0.7959 - val_loss: 0.5180 - val_accuracy: 0.7931
Epoch 12/100
44/44 [==============================] - 1s 27ms/step - loss: 0.4402 - accuracy: 0.8102 - val_loss: 0.4707 - val_accuracy: 0.8013
Epoch 13/100
44/44 [==============================] - 1s 26ms/step - loss: 0.4011 - accuracy: 0.8212 - val_loss: 0.4196 - val_accuracy: 0.8134
Epoch 14/100
44/44 [==============================] - 1s 26ms/step - loss: 0.3749 - accuracy: 0.8198 - val_loss: 0.3780 - val_accuracy: 0.8244
Epoch 15/100
44/44 [==============================] - 1s 27ms/step - loss: 0.3454 - accuracy: 0.8368 - val_loss: 0.3498 - val_accuracy: 0.8400
Epoch 16/100

44/44 [==============================] - 1s 26ms/step - loss: 0.0299 - accuracy: 0.9872 - val_loss: 0.0767 - val_accuracy: 0.9566
Epoch 65/100
44/44 [==============================] - 1s 25ms/step - loss: 0.0284 - accuracy: 0.9908 - val_loss: 0.0702 - val_accuracy: 0.9595
Epoch 66/100
44/44 [==============================] - 1s 26ms/step - loss: 0.0278 - accuracy: 0.9890 - val_loss: 0.0687 - val_accuracy: 0.9570
Epoch 67/100
44/44 [==============================] - 1s 26ms/step - loss: 0.0269 - accuracy: 0.9890 - val_loss: 0.0719 - val_accuracy: 0.9588
Epoch 68/100
44/44 [==============================] - 1s 26ms/step - loss: 0.0261 - accuracy: 0.9908 - val_loss: 0.0712 - val_accuracy: 0.9573
Epoch 69/100
44/44 [==============================] - 1s 26ms/step - loss: 0.0250 - accuracy: 0.9918 - val_loss: 0.0668 - val_accuracy: 0.9613
Epoch 70/100
44/44 [==============================] - 1s 27ms/step - loss: 0.0244 - accuracy: 0.9925 - val_loss: 0.0727 - val_accuracy: 0.9573
Epoch 71/100

In [40]:
results

[0.06409744918346405,
 0.054722417145967484,
 0.07383778691291809,
 0.03491674363613129,
 0.06795434653759003,
 0.06602121889591217,
 0.06763235479593277,
 0.06766890734434128,
 0.0674949362874031,
 0.069367915391922]

In [41]:
np.mean(accs)

0.966755598783493

In [28]:
# 결과 생성
pred_list = []    # 예측 결과를 담을 리스트
for model in models:
  pred = model.predict(series_test)
  pred_list.append(pred)

pred = np.mean(pred_list, axis=0)

In [29]:
# 제출물 생성
sub.iloc[:, 1:] = pred

save_path = os.path.join('sub/test_wieghted.csv')     # 저장 경로
sub.to_csv(save_path, index=False)